In [ ]:
#구글 드라이브 import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
import re
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
import torch


from transformers import BertTokenizer, BertModel

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Euron 6기/team project/preprocessing_final.csv")
df.head()

,title,label,processed_title
0,재학생 신청 및 일정 안내,0,재학생 신청 일정 안내
1,울산대학교 년도 학기 학점교류 수학 안내 만료,0,울산대학교 학점 교류 수학 안내
2,학년도 여름계절제 학부 강의정보 공유를 위한 설문 실시 안내 만료,0,여름 계절제 학부 강의 정보 공유 설문 실시 안내
3,울산과학기술원 학년도 학기 국내대학 학점교류 수학 안내 만료,0,울산과학기술원 국내 대학 학점 교류 수학 안내
4,한국과학기술원 학년도 학기 국내교환 학점교류 수학 안내 만료,0,한국과학기술원 국내 교환 학점 교류 수학 안내


In [ ]:
df.shape

(9423, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9423 entries, 0 to 9422
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            9350 non-null   object
 1   label            9423 non-null   int64 
 2   processed_title  9257 non-null   object
dtypes: int64(1), object(2)
memory usage: 221.0+ KB


전처리 과정에서 title 과 processed_title에 nan값이 생긴 것으로 보임.

In [ ]:
# 결측치 제거
df = df.dropna(subset=['processed_title'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9257 entries, 0 to 9422
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            9257 non-null   object
 1   label            9257 non-null   int64 
 2   processed_title  9257 non-null   object
dtypes: int64(1), object(2)
memory usage: 289.3+ KB


In [ ]:
#불용어 사전
stop_words = ['의','가','이','은','들','는','걍','과','들','과','으로','도','을',
              '를','으로','자','에','와','한','하다','에서','에게', '및', '연도', '년',
              '년도', '학기', 'ㄴ' '학년도', '회', '상반기', '하반기', '년대', '학년',
              '오후', '오전', '오늘', '내일', '회차', '개월', '주년', '종료', 'th', 'st', 'nd', "'s", '접수', '기한', '연장',
              '월', '어', '다', '까지', '제', '등', '등등', '몇', '면', '각', '각각', '마감','공통', '만료',
              '여', '대', '백양']

In [ ]:
# N-그램 벡터화 (1-그램, 2-그램)
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words= stop_words)
X_ngrams = vectorizer.fit_transform(df['processed_title']).toarray()

print(vectorizer.get_feature_names_out())
print(X_ngrams)

['abeek' 'abeek 연구원' 'abeek 자체' ... '희망 청년' '희망자' '희망자 모집']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
# 문장 임베딩 추출
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 데이터를 GPU로 이동
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # 결과를 CPU로 다시 이동

In [ ]:
# 모든 문장에 대해 임베딩 추출
embeddings = [get_sentence_embedding(sentence) for sentence in df['processed_title']]
X_embeddings = pd.DataFrame(embeddings)

In [ ]:
X_embeddings

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.112059,-0.082804,0.027034,-5.452650,0.014368,0.054601,-0.104920,-0.119970,-0.552336,-0.414839,...,0.047894,-0.329213,-0.005385,0.164461,-0.079818,-0.248812,-0.038483,-0.056674,-0.450306,-0.002488
1,0.297481,0.054847,0.049162,-6.170014,-0.001791,0.051517,-0.032435,0.042312,-0.401772,-0.432733,...,0.133794,-0.118319,-0.089400,0.056677,0.048407,-0.178939,-0.131501,-0.033571,-0.400435,-0.007838
2,0.263201,-0.024069,0.152646,-5.348793,-0.014578,0.073721,0.072455,0.063703,-0.098205,-0.370151,...,0.132410,0.021175,0.079008,0.056954,0.024804,-0.198756,-0.193396,0.005567,-0.405065,-0.021561
3,0.146492,-0.026372,-0.040958,-6.181415,0.110575,-0.107565,-0.015185,-0.028272,-0.532167,-0.259729,...,0.083773,0.045090,-0.077636,0.080047,-0.132305,-0.155741,-0.044496,0.080783,-0.399918,-0.008656
4,0.366067,-0.113107,0.027439,-5.782544,-0.038202,0.112120,0.057744,0.020765,-0.189099,-0.272629,...,0.033766,-0.120625,0.046980,0.085250,0.021408,-0.198436,-0.155205,-0.002750,-0.355520,-0.072891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9252,0.285004,0.020230,0.042786,-6.302514,0.008868,0.030297,-0.057990,0.055320,-0.327813,-0.384407,...,0.125787,-0.051898,-0.064771,0.035861,0.092392,-0.153778,-0.172367,-0.015720,-0.376284,0.032971
9253,0.266650,-0.047016,0.049386,-5.273312,0.009486,-0.007374,-0.047152,-0.065066,-0.433100,-0.262979,...,0.144671,-0.240883,-0.189632,0.176070,-0.119640,-0.291074,-0.257411,0.002470,-0.431327,-0.079187
9254,0.307452,-0.192637,-0.106436,-5.425664,0.004087,-0.204730,-0.290625,0.090734,-0.366828,-0.233068,...,0.032486,-0.191160,-0.303442,0.163483,-0.049105,-0.191734,-0.086002,0.149516,-0.314621,-0.105363
9255,0.294305,-0.012400,0.078003,-5.889281,0.006515,-0.020740,0.102940,0.006513,-0.474110,-0.162889,...,0.111665,-0.250946,-0.078004,0.206998,0.059479,-0.270627,0.140353,0.219967,-0.425382,-0.053897


In [ ]:
# 행(row) 단위로 0이 포함된 부분을 확인
rows_with_zeros = X_embeddings[(X_embeddings == 0).any(axis=1)]
rows_with_zeros.shape

(0, 768)

In [ ]:
#중요도 up 단어 플래그
high_priority_words = ['학회', '프로그램', '서포터즈', '인턴', '공모전', '교환', '실습', '취업',
                       '면접', '자소서', '자격증', '고시', '트랙', '교수', '조교', '대학원생', '팀플', '교양', 'ChatGPT', '챗지피티',
                      '계절', '채플', 'mbti', 'MBTI']


df['high_priority_flag'] = df['processed_title'].apply(lambda x: 1 if any(word in x for word in high_priority_words) else 0)
X_high_priority = df['high_priority_flag'].values.reshape(-1, 1)

# 모든 특징 결합
X_combined = np.hstack((X_high_priority, X_ngrams, X_embeddings))

# 라벨
y = df['label'].values


## Random Forest

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=36)

# 가중치 설정
X_train_weighted = np.copy(X_train)
X_test_weighted = np.copy(X_test)

# 중요도를 높이고 싶은 단어 플래그의 가중치를 높임
X_train_weighted[:, 0] *= 2.0  # high_priority_flag의 가중치를 2.0으로 설정
X_test_weighted[:, 0] *= 2.0

# 랜덤 포레스트 모델 학습
RF = RandomForestClassifier(n_estimators=100, random_state=36)
RF.fit(X_train_weighted, y_train)

# 예측 및 평가
y_pred = RF.predict(X_test_weighted)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8396
Precision:  0.8421
Recall:  0.8396
F1 Score:  0.8397
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       972
           1       0.81      0.87      0.84       880

    accuracy                           0.84      1852
   macro avg       0.84      0.84      0.84      1852
weighted avg       0.84      0.84      0.84      1852



In [ ]:
# 훈련 데이터에 대한 예측 및 평가
y_train_pred = RF.predict(X_train_weighted)

print(f'Training Accuracy: {accuracy_score(y_train, y_train_pred): .4f}')
print(f'Training Precision: {precision_score(y_train, y_train_pred, average="weighted"): .4f}')
print(f'Training Recall: {recall_score(y_train, y_train_pred, average="weighted"): .4f}')
print(f'Training F1 Score: {f1_score(y_train, y_train_pred, average="weighted"): .4f}')

Training Accuracy:  0.9733
Training Precision:  0.9733
Training Recall:  0.9733
Training F1 Score:  0.9733


약간의 과적합이 있는 것으로 생각됨. (아마 max_depth 를 지정하지 않아서)

In [ ]:
X_train_weighted.shape

(7405, 9686)

In [ ]:
# 랜덤 포레스트 모델 학습 2
RF_2 = RandomForestClassifier(n_estimators=200, max_depth = 50, min_samples_split = 5, min_samples_leaf = 1, random_state=36)
RF_2.fit(X_train_weighted, y_train)

# 예측 및 평가
y_pred = RF_2.predict(X_test_weighted)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8477
Precision:  0.8510
Recall:  0.8477
F1 Score:  0.8478
              precision    recall  f1-score   support

           0       0.89      0.81      0.85       972
           1       0.81      0.89      0.85       880

    accuracy                           0.85      1852
   macro avg       0.85      0.85      0.85      1852
weighted avg       0.85      0.85      0.85      1852



In [ ]:
# 훈련 데이터에 대한 예측 및 평가
y_train_pred = RF_2.predict(X_train_weighted)

print(f'Training Accuracy: {accuracy_score(y_train, y_train_pred): .4f}')
print(f'Training Precision: {precision_score(y_train, y_train_pred, average="weighted"): .4f}')
print(f'Training Recall: {recall_score(y_train, y_train_pred, average="weighted"): .4f}')
print(f'Training F1 Score: {f1_score(y_train, y_train_pred, average="weighted"): .4f}')

Training Accuracy:  0.9576
Training Precision:  0.9580
Training Recall:  0.9576
Training F1 Score:  0.9576


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

# # 하이퍼파라미터 범위 설정
# param_dist = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, 50, 100, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2']
# }

# # RandomizedSearchCV 설정
# random_search = RandomizedSearchCV(
#     estimator=RandomForestClassifier(random_state=36),
#     param_distributions=param_dist,
#     n_iter=20,  # 시도할 조합의 수
#     cv=3,
#     verbose=2,
#     n_jobs=-1,
#     random_state=36
# )

# # 학습 및 최적 하이퍼파라미터 출력
# random_search.fit(X_train_weighted, y_train)
# print("Best Parameters:", random_search.best_params_)


In [ ]:
# 랜덤 포레스트 모델 학습 3
RF_3 = RandomForestClassifier(n_estimators=300, max_depth = None, min_samples_split = 2, min_samples_leaf = 1,
                                 max_features = 'log2', random_state=36)
RF_3.fit(X_train, y_train)

# 예측 및 평가
y_pred = RF_3.predict(X_test)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8456
Precision:  0.8478
Recall:  0.8456
F1 Score:  0.8457
              precision    recall  f1-score   support

           0       0.88      0.82      0.85       972
           1       0.81      0.87      0.84       880

    accuracy                           0.85      1852
   macro avg       0.85      0.85      0.85      1852
weighted avg       0.85      0.85      0.85      1852



In [ ]:
RF_3 = RandomForestClassifier(n_estimators=300, max_depth = None, min_samples_split = 2, min_samples_leaf = 1,
                                 max_features = 'log2', random_state=36)
RF_3.fit(X_train_weighted, y_train)

# 예측 및 평가
y_pred = RF_3.predict(X_test_weighted)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8456
Precision:  0.8478
Recall:  0.8456
F1 Score:  0.8457
              precision    recall  f1-score   support

           0       0.88      0.82      0.85       972
           1       0.81      0.87      0.84       880

    accuracy                           0.85      1852
   macro avg       0.85      0.85      0.85      1852
weighted avg       0.85      0.85      0.85      1852



In [ ]:
# 훈련 데이터에 대한 예측 및 평가
y_train_pred = RF_3.predict(X_train_weighted)

print(f'Training Accuracy: {accuracy_score(y_train, y_train_pred): .4f}')
print(f'Training Precision: {precision_score(y_train, y_train_pred, average="weighted"): .4f}')
print(f'Training Recall: {recall_score(y_train, y_train_pred, average="weighted"): .4f}')
print(f'Training F1 Score: {f1_score(y_train, y_train_pred, average="weighted"): .4f}')

Training Accuracy:  0.9733
Training Precision:  0.9733
Training Recall:  0.9733
Training F1 Score:  0.9733


## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size = 0.2, random_state = 36)

**Multinomial Naive Bayes**

In [ ]:
# Multinomial Naive Bayes

# 데이터에 음수가 없도록 0과 1사이로 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# 모델 훈련
MNB = MultinomialNB()
MNB.fit(X_train_scaled, y_train)

# 예측
y_pred = MNB.predict(X_test_scaled)

# 평가 지표
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8844
Precision:  0.8962
Recall:  0.8844
F1 Score:  0.8841
              precision    recall  f1-score   support

           0       0.96      0.81      0.88       972
           1       0.82      0.97      0.89       880

    accuracy                           0.88      1852
   macro avg       0.89      0.89      0.88      1852
weighted avg       0.90      0.88      0.88      1852



In [ ]:
# 중요도 높인 버전 - Multinomial Naive Bayes_2

# 중요 단어 가중치 높이기
X_train_weighted = np.copy(X_train_scaled)
X_test_weighted = np.copy(X_test_scaled)

X_train_weighted[:, 0] *= 2.0  # high_priority_flag의 가중치를 2.0으로 설정
X_test_weighted[:, 0] *= 2.0

# 모델 훈련
MNB_2 = MultinomialNB()
MNB_2.fit(X_train_weighted, y_train)

# 예측
y_pred = MNB_2.predict(X_test_weighted)

# 평가 지표
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8850
Precision:  0.8966
Recall:  0.8850
F1 Score:  0.8847
              precision    recall  f1-score   support

           0       0.96      0.81      0.88       972
           1       0.82      0.97      0.89       880

    accuracy                           0.88      1852
   macro avg       0.89      0.89      0.88      1852
weighted avg       0.90      0.88      0.88      1852



**Bernoulli Naive Bayes**

In [ ]:
# Bernoulli Naive Bayes

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# 모델 훈련
BNB = BernoulliNB()
BNB.fit(X_train_scaled, y_train)

# 예측
y_pred = BNB.predict(X_test_scaled)

# 평가 지표
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8942
Precision:  0.9006
Recall:  0.8942
F1 Score:  0.8941
              precision    recall  f1-score   support

           0       0.95      0.84      0.89       972
           1       0.84      0.95      0.90       880

    accuracy                           0.89      1852
   macro avg       0.90      0.90      0.89      1852
weighted avg       0.90      0.89      0.89      1852



In [ ]:
# 중요 단어 가중치 높이기 - Bernoulli Naive Bayes_2

# 중요 단어 가중치 높이기
X_train_weighted = np.copy(X_train_scaled)
X_test_weighted = np.copy(X_test_scaled)

X_train_weighted[:, 0] *= 2.0  # high_priority_flag의 가중치를 2.0으로 설정
X_test_weighted[:, 0] *= 2.0

# 모델 훈련
BNB_2 = BernoulliNB()
BNB_2.fit(X_train_weighted, y_train)

# 예측
y_pred = BNB_2.predict(X_test_weighted)

# 평가 지표
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.8942
Precision:  0.9006
Recall:  0.8942
F1 Score:  0.8941
              precision    recall  f1-score   support

           0       0.95      0.84      0.89       972
           1       0.84      0.95      0.90       880

    accuracy                           0.89      1852
   macro avg       0.90      0.90      0.89      1852
weighted avg       0.90      0.89      0.89      1852



## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=36)

In [ ]:
# X_embeddings만 활용

# 모델 학습
LR = LogisticRegression(max_iter=1000)
LR.fit(X_train, y_train)

# 예측 및 평가
y_pred = LR.predict(X_test)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.7052
Precision:  0.7185
Recall:  0.7052
F1 Score:  0.7033
              precision    recall  f1-score   support

           0       0.78      0.62      0.69       972
           1       0.65      0.80      0.72       880

    accuracy                           0.71      1852
   macro avg       0.72      0.71      0.70      1852
weighted avg       0.72      0.71      0.70      1852



In [ ]:
# X_combined 활용

# 모델 학습
LR_2 = LogisticRegression(max_iter=1000)
LR_2.fit(X_train, y_train)

# 예측 및 평가
y_pred = LR_2.predict(X_test)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.7052
Precision:  0.7185
Recall:  0.7052
F1 Score:  0.7033
              precision    recall  f1-score   support

           0       0.78      0.62      0.69       972
           1       0.65      0.80      0.72       880

    accuracy                           0.71      1852
   macro avg       0.72      0.71      0.70      1852
weighted avg       0.72      0.71      0.70      1852



In [ ]:
# 중요도 높인 버전

# 중요 단어 가중치 높이기
X_train_weighted = np.copy(X_train)
X_test_weighted = np.copy(X_test)

X_train_weighted[:, 0] *= 2.0  # high_priority_flag의 가중치를 2.0으로 설정
X_test_weighted[:, 0] *= 2.0

# 모델 훈련
LR_3 = LogisticRegression(max_iter=1000)
LR_3.fit(X_train_weighted, y_train)

# 예측
y_pred = LR_3.predict(X_test_weighted)

# 평가 지표
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.7041
Precision:  0.7172
Recall:  0.7041
F1 Score:  0.7022
              precision    recall  f1-score   support

           0       0.77      0.62      0.69       972
           1       0.65      0.80      0.72       880

    accuracy                           0.70      1852
   macro avg       0.71      0.71      0.70      1852
weighted avg       0.72      0.70      0.70      1852



Grid Search CV를 활용하여 하이퍼파라미터 튜닝 (정확도를 기준으로 함.)

In [ ]:
'''
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(max_iter=500)

lr_params = {
    'penalty' : ['l1', 'l2'], # 규제 유형
    'C' : [0.01, 0.1, 1, 10], # 규제 강도 (값이 작을수록 강한 규제)
    'solver' : ['lbfgs', 'newton-cg', 'saga'] # 최적화에 사용할 알고리즘
}

grid_search_lr_model = GridSearchCV(model, lr_params, scoring = 'accuracy', cv=3)

grid_search_lr_model.fit(X_train, y_train)
print(grid_search_lr_model.best_params_)
'''

C : 1, penalty : l2, solver : 'lbfgs' 일 때에 가장 높은 정확도 나옴.

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=36)

# 모델 학습
LR_4 = LogisticRegression(max_iter=1000, C=1, penalty='l2', solver='lbfgs')
LR_4.fit(X_train, y_train)

# 예측 및 평가
y_pred = LR_4.predict(X_test)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.9131
Precision:  0.9144
Recall:  0.9131
F1 Score:  0.9131
              precision    recall  f1-score   support

           0       0.94      0.89      0.92       972
           1       0.89      0.94      0.91       880

    accuracy                           0.91      1852
   macro avg       0.91      0.91      0.91      1852
weighted avg       0.91      0.91      0.91      1852



하이퍼 파라미터를 지정하지 않은 모델보다 성능 차이가 없음.

In [ ]:
# 중요도 높인 버전

# 중요 단어 가중치 높이기
X_train_weighted = np.copy(X_train)
X_test_weighted = np.copy(X_test)

X_train_weighted[:, 0] *= 2.0  # high_priority_flag의 가중치를 2.0으로 설정
X_test_weighted[:, 0] *= 2.0

# 모델 훈련
LR_5 = LogisticRegression(max_iter=1000, C=1, penalty='l2', solver='lbfgs')
LR_5.fit(X_train_weighted, y_train)

# 예측
y_pred = LR_5.predict(X_test_weighted)

# 평가 지표
print(f'Accuracy: {accuracy_score(y_test, y_pred): .4f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"): .4f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"): .4f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"): .4f}')
print(classification_report(y_test, y_pred))

Accuracy:  0.9131
Precision:  0.9144
Recall:  0.9131
F1 Score:  0.9131
              precision    recall  f1-score   support

           0       0.94      0.89      0.92       972
           1       0.89      0.94      0.91       880

    accuracy                           0.91      1852
   macro avg       0.91      0.91      0.91      1852
weighted avg       0.91      0.91      0.91      1852

